In [25]:
# ============================================================
# K-League Track1 - BIN Classification + SoftAvg (Flag-Controlled Grid)
# ------------------------------------------------------------
# ✅ You can toggle BIN_Y search and TOPK search independently:
#   - USE_BIN_Y_SEARCH: True -> compare multiple BIN_Y, False -> fixed BIN_Y
#   - USE_TOPK_SEARCH : True -> compare multiple TOPK,  False -> fixed TOPK
#
# ✅ Models: AutoGluon Tabular (LightGBM + CatBoost + Ensemble)
# ✅ Output: final submission csv in required format:
#           game_episode,end_x,end_y
#
# ✅ Selection criterion (proxy):
#   - AutoGluon leaderboard score_val for log_loss is typically NEGATIVE log_loss
#   - Higher (closer to 0) is better
#   - proxy_score = best_score_x + best_score_y (maximize)
#
# ⚠️ You must set BASE_PATH + PATH_COL correctly.
# ============================================================

import os
import json
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from autogluon.tabular import TabularPredictor


# ======================
# 0) USER SETTINGS
# ======================
BASE_PATH = "open_track1/"   # ✅ change to your dataset folder
PATH_COL = "path"            # ✅ test.csv column name containing episode file path

TRAIN_FILE = "train.csv"
TEST_INDEX_FILE = "test.csv"
SAMPLE_SUB_FILE = "sample_submission.csv"
MATCH_INFO_FILE = "match_info.csv"  # optional

PATH_TRAIN = os.path.join(BASE_PATH, TRAIN_FILE)
PATH_TEST_INDEX = os.path.join(BASE_PATH, TEST_INDEX_FILE)
PATH_SAMPLE_SUB = os.path.join(BASE_PATH, SAMPLE_SUB_FILE)
PATH_MATCH_INFO = os.path.join(BASE_PATH, MATCH_INFO_FILE)

# ======================
# EXPERIMENT CONTROL FLAGS (⭐ 핵심)
# ======================
# --- BIN_Y search ---
USE_BIN_Y_SEARCH = True
BIN_Y_CANDIDATES = [5.0, 6.0]
FIXED_BIN_Y = 5.0

# --- TOP-K soft average search ---
USE_TOPK_SEARCH = False
TOPK_CANDIDATES = [1, 3, 5]
FIXED_TOPK = 3

# --- BIN_X fixed (you can extend similarly if you want) ---
BIN_X = 5.0

# --- last K events ---
K = 20

# --- AutoGluon training budget (per model) ---
TIME_LIMIT = 600                # seconds (10min)
PRESETS = "medium_quality"
HYPERPARAMETERS = {"GBM": {}, "CAT": {}}  # LightGBM + CatBoost

# --- Logging / saving ---
PRINT_LEADERBOARD_TOPN = 10
SAVE_ALL_SUBMISSIONS = False    # True -> save every config submission

# --- Final output name ---
FINAL_SUBMISSION_NAME = "submission_FINAL_flagged.csv"


# ======================
# 1) Resolve experiment grid from flags
# ======================
if USE_BIN_Y_SEARCH:
    BIN_Y_LIST = BIN_Y_CANDIDATES
else:
    BIN_Y_LIST = [FIXED_BIN_Y]

if USE_TOPK_SEARCH:
    TOPK_LIST = TOPK_CANDIDATES
else:
    TOPK_LIST = [FIXED_TOPK]

print("🔧 Experiment setup")
print(" - BIN_Y_LIST:", BIN_Y_LIST)
print(" - TOPK_LIST :", TOPK_LIST)
print(" - Total combinations:", len(BIN_Y_LIST) * len(TOPK_LIST))


# ======================
# 2) Path checks
# ======================
for p in [PATH_TRAIN, PATH_TEST_INDEX, PATH_SAMPLE_SUB]:
    if not os.path.exists(p):
        raise FileNotFoundError(f"❌ File not found: {p}")

print("✅ Path OK")
print(" -", PATH_TRAIN)
print(" -", PATH_TEST_INDEX)
print(" -", PATH_SAMPLE_SUB)


# ======================
# 3) Load base CSVs
# ======================
train = pd.read_csv(PATH_TRAIN)
test_index = pd.read_csv(PATH_TEST_INDEX)
sample_sub = pd.read_csv(PATH_SAMPLE_SUB)

match_info = None
if os.path.exists(PATH_MATCH_INFO):
    match_info = pd.read_csv(PATH_MATCH_INFO)

if PATH_COL not in test_index.columns:
    raise KeyError(f"❌ test.csv missing '{PATH_COL}'. Available: {test_index.columns.tolist()}")

print("✅ Loaded index files")
print(" - train shape:", train.shape)
print(" - test_index shape:", test_index.shape)
print(" - sample_sub shape:", sample_sub.shape)


# ======================
# 4) Load all test episode event CSVs
# ======================
test_events_list = []
for _, row in test_index.iterrows():
    rel = str(row[PATH_COL])
    ep_path = os.path.join(BASE_PATH, rel.lstrip("./"))
    if not os.path.exists(ep_path):
        raise FileNotFoundError(f"❌ Missing test episode file: {ep_path}")
    df_ep = pd.read_csv(ep_path)
    test_events_list.append(df_ep)

test_events = pd.concat(test_events_list, ignore_index=True)

# concat train + test
train = train.copy()
test_events = test_events.copy()
train["is_train"] = 1
test_events["is_train"] = 0
events = pd.concat([train, test_events], ignore_index=True)

print(f"✅ Loaded events total={len(events):,} | train={len(train):,} | test={len(test_events):,}")


# ======================
# 5) Basic required columns & sorting
# ======================
REQ_COLS = ["game_episode", "time_seconds", "action_id", "start_x", "start_y", "end_x", "end_y"]
for c in REQ_COLS:
    if c not in events.columns:
        raise KeyError(f"❌ Required column missing: {c}")

events = events.sort_values(["game_episode", "time_seconds", "action_id"]).reset_index(drop=True)

events["event_idx"] = events.groupby("game_episode").cumcount()
events["n_events"] = events.groupby("game_episode")["event_idx"].transform("max") + 1
events["ep_idx_norm"] = events["event_idx"] / (events["n_events"] - 1).clip(lower=1)

# time delta
events["prev_time"] = events.groupby("game_episode")["time_seconds"].shift(1)
events["dt"] = (events["time_seconds"] - events["prev_time"]).fillna(0)

# movement
events["dx"] = events["end_x"] - events["start_x"]
events["dy"] = events["end_y"] - events["start_y"]
events["dist"] = np.sqrt(events["dx"] ** 2 + events["dy"] ** 2)
events["speed"] = events["dist"] / events["dt"].replace(0, 1e-3)

# zones / lanes
events["x_zone"] = (events["start_x"] / (105 / 7)).astype(int).clip(0, 6)
events["lane"] = pd.cut(
    events["start_y"],
    bins=[0, 68 / 3, 2 * 68 / 3, 68],
    labels=[0, 1, 2],
    include_lowest=True
).astype(int)

print("✅ Basic preprocessing done")


# ======================
# 6) Spatial engineered features
# ======================
events["angle_to_goal"] = np.arctan2(34 - events["start_y"], 105 - events["start_x"]) * 180 / np.pi

events["dist_corner_top"] = np.sqrt((105 - events["start_x"]) ** 2 + (0 - events["start_y"]) ** 2)
events["dist_corner_bottom"] = np.sqrt((105 - events["start_x"]) ** 2 + (68 - events["start_y"]) ** 2)
events["dist_to_nearest_corner"] = events[["dist_corner_top", "dist_corner_bottom"]].min(axis=1)

events["is_corner_area"] = (
    (events["start_x"] > 100) &
    ((events["start_y"] < 5) | (events["start_y"] > 63))
).astype(int)

events["angle_goal_x_corner"] = events["angle_to_goal"] * events["is_corner_area"]
events["dist_corner_x_angle"] = events["dist_to_nearest_corner"] * events["angle_to_goal"]

events["dist_to_sideline"] = events["start_y"].apply(lambda y: min(y, 68 - y))
events["angle_to_goal_center"] = np.arctan2(34 - events["start_y"], 105 - events["start_x"])  # radians
events["time_pos_inter"] = events["ep_idx_norm"] * events["start_x"]

print("✅ Spatial features done")


# ======================
# 7) Episode meta from TRAIN last event
# ======================
train_events_only = events[events["is_train"] == 1].copy()
last_events_train = train_events_only.groupby("game_episode", as_index=False).tail(1).copy()

meta_cols = ["game_episode"]
for cand in ["game_id", "team_id", "is_home", "period_id", "time_seconds"]:
    if cand in last_events_train.columns:
        meta_cols.append(cand)

ep_meta = last_events_train[meta_cols].copy()

# derive game_clock_min if possible
if ("period_id" in ep_meta.columns) and ("time_seconds" in ep_meta.columns):
    ep_meta["game_clock_min"] = np.where(
        ep_meta["period_id"] == 1,
        ep_meta["time_seconds"] / 60,
        45 + ep_meta["time_seconds"] / 60
    )
else:
    ep_meta["game_clock_min"] = np.nan

# mark final_team_id if possible
if "team_id" in ep_meta.columns:
    ep_meta = ep_meta.rename(columns={"team_id": "final_team_id"})
    events = events.merge(ep_meta[["game_episode", "final_team_id"]], on="game_episode", how="left")
    if "team_id" in events.columns:
        events["is_final_team"] = (events["team_id"] == events["final_team_id"]).astype(int)
    else:
        events["is_final_team"] = 0
else:
    events["is_final_team"] = 0

print("✅ Episode meta built")


# ======================
# 8) Leakage block (mask last event end_x/end_y and derivatives)
# ======================
events["last_idx"] = events.groupby("game_episode")["event_idx"].transform("max")
events["is_last"] = (events["event_idx"] == events["last_idx"]).astype(int)

mask_last = events["is_last"] == 1
for col in ["end_x", "end_y", "dx", "dy", "dist", "speed"]:
    if col in events.columns:
        events.loc[mask_last, col] = np.nan

print("✅ Leakage blocked (last event end_x/end_y masked)")


# ======================
# 9) Categorical encoding
# ======================
if "type_name" in events.columns:
    events["type_name"] = events["type_name"].fillna("__NA__")
else:
    events["type_name"] = "__NA__"

if "result_name" in events.columns:
    events["result_name"] = events["result_name"].fillna("__NA__")
else:
    events["result_name"] = "__NA__"

events["type_id"] = LabelEncoder().fit_transform(events["type_name"])
events["res_id"] = LabelEncoder().fit_transform(events["result_name"])

if "team_id" in events.columns:
    if events["team_id"].dtype == "object":
        events["team_id_enc"] = LabelEncoder().fit_transform(events["team_id"].astype(str))
    else:
        events["team_id_enc"] = events["team_id"].astype(int)
else:
    events["team_id_enc"] = 0

if "is_home" not in events.columns:
    events["is_home"] = 0
if "period_id" not in events.columns:
    events["period_id"] = 1

print("✅ Categorical encoding done")


# ======================
# 10) Build lastK sequence and pivot to wide table
# ======================
events["rev_idx"] = events.groupby("game_episode")["event_idx"].transform(lambda s: s.max() - s)
lastK = events[events["rev_idx"] < K].copy()

def assign_pos(df):
    df = df.sort_values("event_idx")
    L = len(df)
    df["pos_in_K"] = np.arange(K - L, K)  # left-pad
    return df

lastK = lastK.groupby("game_episode", group_keys=False).apply(assign_pos)

num_cols = [
    "start_x", "start_y", "end_x", "end_y",
    "dx", "dy", "dist", "speed", "dt",
    "ep_idx_norm", "x_zone", "lane",
    "is_final_team",
    "angle_to_goal", "dist_to_nearest_corner", "is_corner_area",
    "angle_goal_x_corner", "dist_corner_x_angle",
    "dist_to_sideline", "angle_to_goal_center", "time_pos_inter"
]
cat_cols = [
    "type_id", "res_id", "team_id_enc",
    "is_home", "period_id", "is_last"
]

wide = lastK[["game_episode", "pos_in_K"] + num_cols + cat_cols].copy()

wide_num = wide.pivot_table(index="game_episode", columns="pos_in_K", values=num_cols, aggfunc="first")
wide_cat = wide.pivot_table(index="game_episode", columns="pos_in_K", values=cat_cols, aggfunc="first")

wide_num.columns = [f"{c}_{int(p)}" for c, p in wide_num.columns]
wide_cat.columns = [f"{c}_{int(p)}" for c, p in wide_cat.columns]

X_base = pd.concat([wide_num, wide_cat], axis=1).reset_index()

# attach ep_meta
meta_use = ["game_episode", "game_clock_min"]
for c in ["game_id", "final_team_id", "is_home", "period_id"]:
    if c in ep_meta.columns:
        meta_use.append(c)

X_base = X_base.merge(ep_meta[meta_use], on="game_episode", how="left")
X_base = X_base.fillna(0)

# train mask by episodes that exist in train labels (train last events)
train_eps = last_events_train["game_episode"].unique()
train_mask = X_base["game_episode"].isin(train_eps)

print("✅ Wide features built")
print(" - X_base shape:", X_base.shape)
print(" - Train episodes:", train_mask.sum(), "| Test episodes:", (~train_mask).sum())


# ======================
# 11) Utility functions
# ======================
def proba_to_coord_topk_softavg(proba: np.ndarray, bin_size: float, top_k: int) -> float:
    """
    Convert class probability vector into a continuous coordinate:
      - top_k=1: argmax bin center
      - top_k>1: weighted average of top-k bin centers by probability
    """
    if top_k <= 1:
        idx = int(np.argmax(proba))
        return idx * bin_size + bin_size / 2

    top_idx = np.argsort(proba)[-top_k:]
    weights = proba[top_idx]
    wsum = weights.sum()
    if wsum <= 0:
        idx = int(np.argmax(proba))
        return idx * bin_size + bin_size / 2

    weights = weights / wsum
    coords = top_idx * bin_size + bin_size / 2
    return float(np.sum(weights * coords))

def best_score_val(predictor: TabularPredictor) -> float:
    """
    Return top row score_val from leaderboard.
    For log_loss, AutoGluon often reports negative log_loss, so higher is better.
    """
    lb = predictor.leaderboard(silent=True)
    return float(lb.iloc[0]["score_val"])


# ======================
# 12) Main experiment loop (flag-controlled)
# ======================
results = []
best_rec = None

for BIN_Y in BIN_Y_LIST:
    NX = int(np.ceil(105 / BIN_X))
    NY = int(np.ceil(68 / BIN_Y))

    # labels for this BIN_Y
    last_local = last_events_train.copy()
    last_local["target_x_bin"] = (last_local["end_x"] / BIN_X).astype(int).clip(0, NX - 1)
    last_local["target_y_bin"] = (last_local["end_y"] / BIN_Y).astype(int).clip(0, NY - 1)
    labels_local = last_local[["game_episode", "target_x_bin", "target_y_bin"]].copy()

    X_all = X_base.merge(labels_local, on="game_episode", how="left")

    X_train = X_all[train_mask].copy()
    X_test = X_all[~train_mask].copy()

    y_train_x = X_train["target_x_bin"].astype(int)
    y_train_y = X_train["target_y_bin"].astype(int)

    drop_cols = ["game_episode", "target_x_bin", "target_y_bin"]
    if "game_id" in X_train.columns:
        drop_cols.append("game_id")

    X_train_feat = X_train.drop(columns=drop_cols, errors="ignore").fillna(0)
    X_test_feat = X_test.drop(columns=drop_cols, errors="ignore").fillna(0)

    # train frames
    train_x = X_train_feat.copy()
    train_x["target_x_bin"] = y_train_x

    train_y = X_train_feat.copy()
    train_y["target_y_bin"] = y_train_y

    # unique model path per BIN_Y config
    by_tag = str(BIN_Y).replace(".", "p")
    model_path_x = f"ag_models_xbin_BY{by_tag}"
    model_path_y = f"ag_models_ybin_BY{by_tag}"

    print("\n" + "=" * 90)
    print(f"🚀 TRAIN BIN_Y={BIN_Y} (NX={NX}, NY={NY}) | time_limit={TIME_LIMIT}s/model")
    print("=" * 90)

    predictor_x = TabularPredictor(
        label="target_x_bin",
        problem_type="multiclass",
        eval_metric="log_loss",
        path=model_path_x
    ).fit(
        train_data=train_x,
        time_limit=TIME_LIMIT,
        presets=PRESETS,
        hyperparameters=HYPERPARAMETERS,
        verbosity=2
    )

    predictor_y = TabularPredictor(
        label="target_y_bin",
        problem_type="multiclass",
        eval_metric="log_loss",
        path=model_path_y
    ).fit(
        train_data=train_y,
        time_limit=TIME_LIMIT,
        presets=PRESETS,
        hyperparameters=HYPERPARAMETERS,
        verbosity=2
    )

    # leaderboard
    lbx = predictor_x.leaderboard(silent=True)
    lby = predictor_y.leaderboard(silent=True)

    print("\n[X-bin leaderboard head]")
    print(lbx.head(PRINT_LEADERBOARD_TOPN))
    print("\n[Y-bin leaderboard head]")
    print(lby.head(PRINT_LEADERBOARD_TOPN))

    score_x = best_score_val(predictor_x)
    score_y = best_score_val(predictor_y)
    proxy_score = score_x + score_y  # maximize (higher is better)

    print(f"\n✅ Proxy score = score_x + score_y = {score_x:.6f} + {score_y:.6f} = {proxy_score:.6f}")

    # predict proba once per BIN_Y
    proba_x = predictor_x.predict_proba(X_test_feat).values
    proba_y = predictor_y.predict_proba(X_test_feat).values

    # TOPK loop (flag-controlled list)
    for TOPK in TOPK_LIST:
        pred_x = np.array([proba_to_coord_topk_softavg(p, BIN_X, TOPK) for p in proba_x])
        pred_y = np.array([proba_to_coord_topk_softavg(p, BIN_Y, TOPK) for p in proba_y])

        pred_x = np.clip(pred_x, 0, 105)
        pred_y = np.clip(pred_y, 0, 68)

        # build submission in correct format without merge suffix
        pred_df = X_test[["game_episode"]].copy()
        pred_df["end_x"] = pred_x
        pred_df["end_y"] = pred_y

        sub = sample_sub.copy().set_index("game_episode")
        pred_df = pred_df.set_index("game_episode")

        sub["end_x"] = pred_df["end_x"]
        sub["end_y"] = pred_df["end_y"]
        sub = sub.reset_index()[["game_episode", "end_x", "end_y"]]

        out_name = f"submission_BY{by_tag}_TOPK{TOPK}.csv"
        if SAVE_ALL_SUBMISSIONS:
            sub.to_csv(out_name, index=False)

        rec = {
            "BIN_X": BIN_X,
            "BIN_Y": BIN_Y,
            "NX": NX,
            "NY": NY,
            "TOPK": TOPK,
            "score_x": score_x,
            "score_y": score_y,
            "proxy_score": proxy_score,
            "pred_end_x_mean": float(np.mean(pred_x)),
            "pred_end_y_mean": float(np.mean(pred_y)),
            "model_path_x": model_path_x,
            "model_path_y": model_path_y,
            "submission_name": out_name if SAVE_ALL_SUBMISSIONS else None,
        }
        results.append(rec)

        # choose best: maximize proxy_score; tie -> prefer TOPK=3 if present
        if best_rec is None:
            best_rec = rec
        else:
            if rec["proxy_score"] > best_rec["proxy_score"] + 1e-12:
                best_rec = rec
            elif abs(rec["proxy_score"] - best_rec["proxy_score"]) <= 1e-12:
                if (best_rec["TOPK"] != 3) and (rec["TOPK"] == 3):
                    best_rec = rec

print("\n" + "=" * 100)
print("🏆 BEST CONFIG (Flagged Grid)")
print("=" * 100)
print(json.dumps(best_rec, indent=2, ensure_ascii=False))


# ======================
# 13) Train best config again & save FINAL submission
# ======================
BEST_BIN_Y = best_rec["BIN_Y"]
BEST_TOPK = best_rec["TOPK"]

NX = int(np.ceil(105 / BIN_X))
NY = int(np.ceil(68 / BEST_BIN_Y))

print("\n" + "=" * 90)
print(f"🔁 Re-train BEST for final submission: BIN_Y={BEST_BIN_Y}, TOPK={BEST_TOPK}")
print("=" * 90)

# rebuild labels for best BIN_Y
last_local = last_events_train.copy()
last_local["target_x_bin"] = (last_local["end_x"] / BIN_X).astype(int).clip(0, NX - 1)
last_local["target_y_bin"] = (last_local["end_y"] / BEST_BIN_Y).astype(int).clip(0, NY - 1)
labels_local = last_local[["game_episode", "target_x_bin", "target_y_bin"]].copy()

X_all = X_base.merge(labels_local, on="game_episode", how="left")
X_train = X_all[train_mask].copy()
X_test = X_all[~train_mask].copy()

y_train_x = X_train["target_x_bin"].astype(int)
y_train_y = X_train["target_y_bin"].astype(int)

drop_cols = ["game_episode", "target_x_bin", "target_y_bin"]
if "game_id" in X_train.columns:
    drop_cols.append("game_id")

X_train_feat = X_train.drop(columns=drop_cols, errors="ignore").fillna(0)
X_test_feat = X_test.drop(columns=drop_cols, errors="ignore").fillna(0)

train_x = X_train_feat.copy()
train_x["target_x_bin"] = y_train_x
train_y = X_train_feat.copy()
train_y["target_y_bin"] = y_train_y

best_tag = str(BEST_BIN_Y).replace(".", "p")
model_path_x = f"ag_models_xbin_BEST_BY{best_tag}"
model_path_y = f"ag_models_ybin_BEST_BY{best_tag}"

predictor_x = TabularPredictor(
    label="target_x_bin",
    problem_type="multiclass",
    eval_metric="log_loss",
    path=model_path_x
).fit(
    train_data=train_x,
    time_limit=TIME_LIMIT,
    presets=PRESETS,
    hyperparameters=HYPERPARAMETERS,
    verbosity=2
)

predictor_y = TabularPredictor(
    label="target_y_bin",
    problem_type="multiclass",
    eval_metric="log_loss",
    path=model_path_y
).fit(
    train_data=train_y,
    time_limit=TIME_LIMIT,
    presets=PRESETS,
    hyperparameters=HYPERPARAMETERS,
    verbosity=2
)

# proba -> softavg coords
proba_x = predictor_x.predict_proba(X_test_feat).values
proba_y = predictor_y.predict_proba(X_test_feat).values

pred_x = np.array([proba_to_coord_topk_softavg(p, BIN_X, BEST_TOPK) for p in proba_x])
pred_y = np.array([proba_to_coord_topk_softavg(p, BEST_BIN_Y, BEST_TOPK) for p in proba_y])

pred_x = np.clip(pred_x, 0, 105)
pred_y = np.clip(pred_y, 0, 68)

pred_df = X_test[["game_episode"]].copy()
pred_df["end_x"] = pred_x
pred_df["end_y"] = pred_y

sub = sample_sub.copy().set_index("game_episode")
pred_df = pred_df.set_index("game_episode")

sub["end_x"] = pred_df["end_x"]
sub["end_y"] = pred_df["end_y"]
sub = sub.reset_index()[["game_episode", "end_x", "end_y"]]

sub.to_csv(FINAL_SUBMISSION_NAME, index=False)

print("\n✅ FINAL submission saved:", FINAL_SUBMISSION_NAME)
print(sub.head(10))
print("✅ FINAL columns:", sub.columns.tolist())


🔧 Experiment setup
 - BIN_Y_LIST: [5.0, 6.0]
 - TOPK_LIST : [3]
 - Total combinations: 2
✅ Path OK
 - open_track1/train.csv
 - open_track1/test.csv
 - open_track1/sample_submission.csv
✅ Loaded index files
 - train shape: (356721, 15)
 - test_index shape: (2414, 3)
 - sample_sub shape: (2414, 3)
✅ Loaded events total=409,831 | train=356,721 | test=53,110
✅ Basic preprocessing done
✅ Spatial features done
✅ Episode meta built
✅ Leakage blocked (last event end_x/end_y masked)
✅ Categorical encoding done


/var/folders/8g/x3kqv_gx5hdb25l1fv6qd8jw0000gn/T/ipykernel_49695/4259314962.py:307: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  lastK = lastK.groupby("game_episode", group_keys=False).apply(assign_pos)
/var/folders/8g/x3kqv_gx5hdb25l1fv6qd8jw0000gn/T/ipykernel_49695/4259314962.py:340: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_base = X_base.fillna(0)
/var/folders/8g/x3kqv_gx5hdb25l1fv6qd8jw0000gn/T/ipykernel_49695/4259314962.py:412: FutureWarning: Downcasting object dtype arrays

✅ Wide features built
 - X_base shape: (17849, 540)
 - Train episodes: 15435 | Test episodes: 2414

🚀 TRAIN BIN_Y=5.0 (NX=21, NY=14) | time_limit=600s/model


Beginning AutoGluon training ... Time limit = 600s
AutoGluon will save models to "/Users/yangjinmo/Desktop/k_league_ml/ag_models_xbin_BY5p0"
Train Data Rows:    15435
Train Data Columns: 538
Label Column:       target_x_bin
Problem Type:       multiclass
Preprocessing data ...
Train Data Class Count: 21
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    3291.89 MB
	Train Data (Original)  Memory Usage: 70.98 MB (2.2% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 64 features to boolean dtype as they only contain 2 unique values.
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:


[X-bin leaderboard head]
                 model  score_val eval_metric  pred_time_val    fit_time  \
0  WeightedEnsemble_L2  -2.104453    log_loss       0.045542  381.736680   
1             CatBoost  -2.119763    log_loss       0.012725  302.091879   
2             LightGBM  -2.155898    log_loss       0.031681   79.590763   

   pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  \
0                0.001136           0.054038            2       True   
1                0.012725         302.091879            1       True   
2                0.031681          79.590763            1       True   

   fit_order  
0          3  
1          2  
2          1  

[Y-bin leaderboard head]
                 model  score_val eval_metric  pred_time_val    fit_time  \
0  WeightedEnsemble_L2  -1.777630    log_loss       0.040382  372.208516   
1             CatBoost  -1.792829    log_loss       0.011853  317.810023   
2             LightGBM  -1.808585    log_loss       0.026599   54.

...


🚀 TRAIN BIN_Y=6.0 (NX=21, NY=12) | time_limit=600s/model


	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 64 features to boolean dtype as they only contain 2 unique values.
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Useless Original Features (Count: 21): ['is_final_team_19', 'is_last_0', 'is_last_1', 'is_last_2', 'is_last_3', 'is_last_4', 'is_last_5', 'is_last_6', 'is_last_7', 'is_last_8', 'is_last_9', 'is_last_10', 'is_last_11', 'is_last_12', 'is_last_13', 'is_last_14', 'is_last_15', 'is_last_16', 'is_last_17', 'is_last_18', 'is_last_19']
		These features carry no predictive signal and should be manually investigated.
		This is typically a feature which has the same value for all rows.
		These features do not need to be present at inference time.
	Unused Original Features (Count: 5): ['time_pos_inter_19', 'type_id_19


[X-bin leaderboard head]
                 model  score_val eval_metric  pred_time_val    fit_time  \
0  WeightedEnsemble_L2  -2.104453    log_loss       0.037212  373.839991   
1             CatBoost  -2.119763    log_loss       0.011246  298.322105   
2             LightGBM  -2.155898    log_loss       0.024983   75.468402   

   pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  \
0                0.000984           0.049483            2       True   
1                0.011246         298.322105            1       True   
2                0.024983          75.468402            1       True   

   fit_order  
0          3  
1          2  
2          1  

[Y-bin leaderboard head]
                 model  score_val eval_metric  pred_time_val    fit_time  \
0  WeightedEnsemble_L2  -1.658445    log_loss       0.033179  255.109275   
1             CatBoost  -1.667581    log_loss       0.008859  196.630645   
2             LightGBM  -1.685180    log_loss       0.023209   58.

/var/folders/8g/x3kqv_gx5hdb25l1fv6qd8jw0000gn/T/ipykernel_49695/4259314962.py:563: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_train_feat = X_train.drop(columns=drop_cols, errors="ignore").fillna(0)
/var/folders/8g/x3kqv_gx5hdb25l1fv6qd8jw0000gn/T/ipykernel_49695/4259314962.py:564: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_test_feat = X_test.drop(columns=drop_cols, errors="ignore").fillna(0)
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.5.0
Python Version:     3.13.9
Operating System:


✅ FINAL submission saved: submission_FINAL_flagged.csv
  game_episode      end_x      end_y
0     153363_1  66.803431   5.939561
1     153363_2  37.273295  62.813239
2     153363_6  35.577678  64.887259
3     153363_7  52.682555   5.903442
4     153363_8  82.271261   8.201394
5     153363_9  76.906888  68.000000
6    153363_10  57.111227   6.912072
7    153363_12  66.878253   4.290965
8    153363_13  32.663202  65.978741
9    153363_15  59.366657   5.604907
✅ FINAL columns: ['game_episode', 'end_x', 'end_y']


In [26]:
# =======================
# 결과 확인 및 3단계 검증법
# (BIN Classification + Soft Average 버전)
# 셀 0의 베스트 설정에 맞춰서 검증
# =======================

from autogluon.tabular import TabularPredictor
import pandas as pd
import numpy as np
import json

# =======================
# 실행 플래그
# =======================
RUN_STEP1_FEATURE_IMPORTANCE = False   # ⚠️ 매우 느림
RUN_STEP2_LEADERBOARD = True
RUN_STEP3_SITUATIONAL_ANALYSIS = True

# =======================
# 모델 경로 & 제출 파일 (셀 0의 베스트 설정 사용)
# =======================
# 셀 0에서 생성된 베스트 모델 경로 사용
# 베스트 설정이 이미 학습되어 있다고 가정
# 만약 베스트 모델이 없다면, 셀 0의 results에서 최신 모델 경로 사용

# 베스트 모델 경로 (셀 0의 best_rec에서 가져오거나 직접 지정)
# 셀 0 실행 후 best_rec 변수에서 가져올 수 있음
try:
    # 셀 0에서 best_rec이 정의되어 있다면 사용
    best_tag = str(best_rec["BIN_Y"]).replace(".", "p")
    model_path_x = f"ag_models_xbin_BEST_BY{best_tag}"
    model_path_y = f"ag_models_ybin_BEST_BY{best_tag}"
    submission_filename = FINAL_SUBMISSION_NAME
except NameError:
    # 셀 0이 실행되지 않았다면 기본값 사용 (사용자가 수동으로 지정)
    print("⚠️ 셀 0의 best_rec을 찾을 수 없습니다. 기본 경로를 사용합니다.")
    print("   셀 0을 먼저 실행하거나, 아래 경로를 수동으로 설정하세요.")
    # 예시: BIN_Y=5.0인 경우
    model_path_x = "ag_models_xbin_BEST_BY5p"
    model_path_y = "ag_models_ybin_BEST_BY5p"
    submission_filename = "submission_FINAL_flagged.csv"

print(f"모델 경로: {model_path_x}, {model_path_y}")
print(f"제출 파일: {submission_filename}")

# =======================
# 모델 로드
# =======================
try:
    predictor_x = TabularPredictor.load(model_path_x)
    predictor_y = TabularPredictor.load(model_path_y)
    print("✓ 모델 로드 완료")
except Exception as e:
    print(f"❌ 모델 로드 실패: {e}")
    print("   셀 0을 먼저 실행하여 모델을 학습하세요.")
    raise

print("=" * 70)
print("3단계 검증법 (BIN + SoftAvg)")
print("=" * 70)

# ============================================================
# 1단계: Feature Importance (참고용)
# ============================================================
if RUN_STEP1_FEATURE_IMPORTANCE:
    print("\n[1단계] Feature Importance 확인")

    print("\n[X-bin 모델]")
    fi_x = predictor_x.feature_importance(data=X_train_feat)
    print(fi_x.iloc[:, 0].sort_values(ascending=False).head(20))

    print("\n[Y-bin 모델]")
    fi_y = predictor_y.feature_importance(data=X_train_feat)
    print(fi_y.iloc[:, 0].sort_values(ascending=False).head(20))

else:
    print("\n1단계: Feature Importance 건너뜀")

# ============================================================
# 2단계: Leaderboard (log_loss 기준)
# ============================================================
if RUN_STEP2_LEADERBOARD:
    print("\n" + "=" * 70)
    print("[2단계] Leaderboard (log_loss 기준)")
    print("=" * 70)

    print("\n[X-bin 모델]")
    lb_x = predictor_x.leaderboard(silent=True)
    print(lb_x.head(5))
    print(f"✓ Best log_loss (X): {lb_x.iloc[0]['score_val']:.5f}")

    print("\n[Y-bin 모델]")
    lb_y = predictor_y.leaderboard(silent=True)
    print(lb_y.head(5))
    print(f"✓ Best log_loss (Y): {lb_y.iloc[0]['score_val']:.5f}")

    # ⚠️ Train accuracy는 참고용 (RMSE 대회)
    print("\n[Train Accuracy 참고용]")
    pred_x_train = predictor_x.predict(X_train_feat)
    pred_y_train = predictor_y.predict(X_train_feat)

    acc_x = (pred_x_train == y_train_x).mean()
    acc_y = (pred_y_train == y_train_y).mean()

    print(f"Train Accuracy X-bin: {acc_x:.4f}")
    print(f"Train Accuracy Y-bin: {acc_y:.4f}")

else:
    print("\n2단계: Leaderboard 건너뜀")

# ============================================================
# 3단계: 상황별 예측 분포 분석 (⭐ 핵심)
# ============================================================
if RUN_STEP3_SITUATIONAL_ANALYSIS:
    print("\n" + "=" * 70)
    print("[3단계] 상황별 예측 분포 분석")
    print("=" * 70)

    # 제출 파일 로드
    try:
        sub = pd.read_csv(submission_filename)
    except FileNotFoundError:
        print(f"⚠️ 제출 파일을 찾을 수 없습니다: {submission_filename}")
        print("   셀 0을 먼저 실행하여 제출 파일을 생성하세요.")
        sub = None

    if sub is not None:
        # --------------------------------------------------------
        # X_test에서 존재하는 컬럼만 안전하게 선택
        # --------------------------------------------------------
        merge_cols = ["game_episode"]

        if "period_id" in X_test.columns:
            merge_cols.append("period_id")

        if "game_clock_min" in X_test.columns:
            merge_cols.append("game_clock_min")

        # 병합
        merged = X_test[merge_cols].merge(
            sub, on="game_episode", how="left"
        )

        # --------------------------------------------------------
        # 컬럼 존재 sanity check
        # --------------------------------------------------------
        if ("end_x" not in merged.columns) or ("end_y" not in merged.columns):
            raise ValueError(
                f"❌ 제출 파일 컬럼 오류\n"
                f"merged columns: {merged.columns.tolist()}"
            )

        # --------------------------------------------------------
        # 전체 분포
        # --------------------------------------------------------
        print("\n[전체 예측 좌표 통계]")
        print(merged[["end_x", "end_y"]].describe())

        # --------------------------------------------------------
        # 전 / 후반 비교
        # --------------------------------------------------------
        if "period_id" in merged.columns:
            first_half = merged[merged["period_id"] == 1]
            second_half = merged[merged["period_id"] == 2]

            if len(first_half) > 0:
                print("\n[전반전 예측 분포]")
                print(first_half[["end_x", "end_y"]].describe())

            if len(second_half) > 0:
                print("\n[후반전 예측 분포]")
                print(second_half[["end_x", "end_y"]].describe())

        # --------------------------------------------------------
        # 공격 방향 sanity check (X축)
        # --------------------------------------------------------
        mean_x = merged["end_x"].mean()
        print("\n[공격 방향 sanity check]")
        print(
            f"end_x 평균: {mean_x:.2f} "
            "(값이 클수록 전진 패스 성향)"
        )

        # --------------------------------------------------------
        # Y축 중앙 쏠림 sanity check
        # --------------------------------------------------------
        std_y = merged["end_y"].std()
        print("\n[Y축 분산 sanity check]")
        print(
            f"end_y 표준편차: {std_y:.2f} "
            "(너무 작으면 중앙 쏠림 의심)"
        )

        # --------------------------------------------------------
        # 시간대별 (선택)
        # --------------------------------------------------------
        if "game_clock_min" in merged.columns:
            early = merged[merged["game_clock_min"] < 30]
            late = merged[merged["game_clock_min"] > 75]

            if len(early) > 0:
                print("\n[초반 (0~30분)]")
                print(early[["end_x", "end_y"]].describe())

            if len(late) > 0:
                print("\n[후반 막판 (75분~)]")
                print(late[["end_x", "end_y"]].describe())

else:
    print("\n3단계: 상황 분석 건너뜀")

print("\n" + "=" * 70)
print("검증 완료 (BIN + SoftAvg)")
print("=" * 70)




모델 경로: ag_models_xbin_BEST_BY6p0, ag_models_ybin_BEST_BY6p0
제출 파일: submission_FINAL_flagged.csv
✓ 모델 로드 완료
3단계 검증법 (BIN + SoftAvg)

1단계: Feature Importance 건너뜀

[2단계] Leaderboard (log_loss 기준)

[X-bin 모델]
                 model  score_val eval_metric  pred_time_val    fit_time  \
0  WeightedEnsemble_L2  -2.104453    log_loss       0.041228  376.767408   
1             CatBoost  -2.119763    log_loss       0.010085  299.704512   
2             LightGBM  -2.155898    log_loss       0.030132   77.010774   

   pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  \
0                0.001011           0.052122            2       True   
1                0.010085         299.704512            1       True   
2                0.030132          77.010774            1       True   

   fit_order  
0          3  
1          2  
2          1  
✓ Best log_loss (X): -2.10445

[Y-bin 모델]
                 model  score_val eval_metric  pred_time_val    fit_time  \
0  WeightedEnsemble_L2 